<a href="https://colab.research.google.com/github/rebeccaastaix/QM2/blob/main/Gender_Assignemnt_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install gender-guesser

In [11]:
from google.colab import files
uploaded = files.upload()

Saving 1-12 MAIN-Curated-dataset-(4086-records).xlsx to 1-12 MAIN-Curated-dataset-(4086-records) (1).xlsx


In [13]:
df = pd.read_excel("1-12 MAIN-Curated-dataset-(4086-records).xlsx")
df.columns.tolist()

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['Column',
 'ARK',
 'URL',
 'Type of resource',
 'Content type',
 'Material type',
 'LC number',
 'OCLC number',
 'Name',
 'Dates associated with name',
 'Type of name',
 'Role',
 'All names',
 'Title',
 'Uniform title',
 'Variant titles',
 'Series title',
 'Number within series',
 'Country of publication',
 '(edited and completed) Country of publication',
 'Place of publication',
 '(edited) Place of publication',
 'Publisher',
 'Date of publication (standardised)',
 'Date of publication (not standardised)',
 'Edition',
 'Physical description',
 'Dewey classification',
 'BL shelfmark',
 'Topics',
 'Genre',
 'Target audience',
 'Literary form',
 'Languages',
 '(edited) Languages',
 'Language of original',
 'Language of intermediate translations',
 'Language notes',
 'Contents',
 'Abstract',
 'Notes']

In [15]:
import pandas as pd
import re
import numpy as np

# Helper lists
titles = {'mr', 'mrs', 'ms', 'miss', 'sir', 'dr', 'prof', 'madame', 'mme',
          'm.', 'lord', 'lady', 'baron', 'baronne', 'count', 'conte', 'don', 'dame'}
particles = {'de', 'du', 'von', 'van', 'da', 'del', 'della', 'la', 'le', 'des'}

def extract_first_name(name):
    """
    Try to extract a usable first name from the 'Name' column.
    Returns a string like 'Alfred' or None if we can't find one.
    """
    if not isinstance(name, str):
        return None

    s = name.strip()
    if not s:
        return None

    # 1) If there is something in parentheses, it is often the given name:
    # Example: "Jerningham, mr (Edward)" -> Edward
    m = re.search(r'\(([^)]+)\)', s)
    if m:
        inside = m.group(1)
        tokens = re.split(r'[,\s]+', inside)
        for t in tokens:
            t0 = t.strip()
            t_clean = t0.strip(".").lower()
            if (not t_clean or t_clean in titles or t_clean in particles
                or len(t_clean) <= 1 or not t0[0].isupper()):
                continue
            return t_clean.capitalize()

    # 2) If there is a comma, pattern is usually "Last, First ..."
    # Example: "Tennyson, Alfred Tennyson, Baron" -> Alfred
    had_comma = ',' in s
    if had_comma:
        after = s.split(',', 1)[1]  # take part after first comma
        tokens = re.split(r'[,\s]+', after.replace('-', ' '))
        for t in tokens:
            t0 = t.strip()
            t_clean = t0.strip(".").lower()
            if (not t_clean or t_clean in titles or t_clean in particles
                or len(t_clean) <= 1 or t0.endswith('.') or len(t0) == 1
                or not t0[0].isupper()):
                continue
            return t_clean.capitalize()
        # If we had a comma but still didn't find a good first name, give up
        return None

    # 3) Otherwise assume "First Last" and take first capitalised word
    tokens = re.split(r'\s+', s.replace('-', ' '))
    for t in tokens:
        t0 = t.strip()
        t_clean = t0.strip(".,").lower()
        if (not t_clean or t_clean in titles or t_clean in particles
            or len(t_clean) <= 1 or not t0[0].isupper()):
            continue
        return t_clean.capitalize()

    return None


In [17]:
df['FirstName'] = df['Name'].apply(extract_first_name)
df[['Name', 'FirstName']].head(1000)

,Name,FirstName
0,"Tennyson, Alfred Tennyson, Baron",Alfred
1,"Pope, Alexander",Alexander
2,"Jerningham, mr (Edward)",Edward
3,"Bouilly, Jean Nicolas",Jean
4,NaN,None
...,...,...
995,Koninklijk Instituut van Ingenieurs (Hague),Hague
996,"Haigneré, Daniel",Daniel
997,"Hanauer, Charles Auguste",Charles
998,"Hardivillier, Harles Achille d'",Harles


No charts were generated by quickchart


In [18]:
!pip install gender-guesser
from gender_guesser.detector import Detector

detector = Detector()

def guess_gender(first):
    if pd.isna(first) or first is None:
        return 'Unknown'
    g = detector.get_gender(first)
    mapping = {
        'male': 'Male',
        'mostly_male': 'Male',
        'female': 'Female',
        'mostly_female': 'Female',
        'andy': 'Unknown',
        'unknown': 'Unknown'
    }
    return mapping.get(g, 'Unknown')

df['Gender'] = df['FirstName'].apply(guess_gender)
df[['Name', 'FirstName', 'Gender']].head(20)

,Name,FirstName,Gender
0,"Tennyson, Alfred Tennyson, Baron",Alfred,Male
1,"Pope, Alexander",Alexander,Male
2,"Jerningham, mr (Edward)",Edward,Male
3,"Bouilly, Jean Nicolas",Jean,Male
4,NaN,None,Unknown
5,"Cossé, P., satirist",None,Unknown
6,"Driver, F. W.",None,Unknown
7,"Gaubil, Antoine",Antoine,Male
8,NaN,None,Unknown
9,"Petit, Charles, traveller",Charles,Male


In [19]:
df.to_excel("dataset_with_gender.xlsx", index=False)

In [20]:
from google.colab import files
files.download("dataset_with_gender.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>